### Сторонние библиотеки использовать нельзя

### Задача 0 [Библиотека] (0.15 балла)  

**Условие:** 


В библиотеке хранятся книги и журналы. У каждой сущности есть общие характеристики, такие как: название, автор, жанр, число страниц, формат страниц, индекс редкости (от 1 до 10) и текст. Также у разных сущностей могут быть свои атрибуты. Хочется все редкие издания (индекс 9 или 10) дополнительно сохранять в некое хранилище (пусть json-файл), а также хочется понимать какую площадь занимает издание, если разложить все его страницы на полу.     


**Комментарий:**

Это задача с семинара на организацию иерархии классов. Идея в том, что нужно разделять сущности в зависимости от их применения. Например, есть книга как некий абстрактный объект, а есть библиотечная книга, у которой есть свои особенности. Также для сохранения книг в json нужно использвать классы-примеси.


Иерархия классов:

In [74]:
PAGES_FORMAT = {
    'A1': (2048, 1024),
    'A2': (1024, 512),
    'A3': (512, 256),
    'A4': (297, 210),
}


class ReadableEntity:
    pass


class Journal(ReadableEntity):
    pass


class Book(ReadableEntity):
    pass


class Exporter:
    
    def export_to_txt(self, file_path):
        with open(file_path, 'w') as f:
            for key in self.__dict__:
                f.write("{}: {}".format(key, self.__dict__[key]))
     
    
class LibraryJournal(Journal, Exporter):
    pass


class LibraryBook(Book, Exporter):
    pass

### Задача 1 [Размер объектов] (0 - 0.15 балла)  

**Условие:** 

Написать функцию получения реального объема занимаемой объектом памяти объектом. 


1) Для int, str, list, tuple, dict **(0.05 балла)**

2) Для всех типов **(+0.1 балла)**


**Комментарий:**

На занятиях не раз говорилось, что `sys.getsizeof` умеет находить размер простых объектов, но если речь идет об объектах, вроде list, то функция вернет не совсем то, что может ожидать разработчик, потому что список хранит указатели на объекты. 

*Пример:*
```
sys.getsizeof([]) == 64
sys.getsizeof(['aaaaaaa']) == 72
```
Но
```
sys.getsizeof('aaaaaaa') == 56
```


### Задача 2 [Многочлены] (0.64 балла)

**Условие:**

Реализовать класс многочлена. Определить операции:

1) *сложения* - **(0.02 балла)**

2) *вычитания* - **(0.02 балла)**

3) *умножения* - **(0.04 балла)**

3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

4) *деления* - **(0.05 балла)**

5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**

6) *представления многочлена в человеческом виде* - **(0.02 балла)**

7) *дифференцирования* - **(0.05 балла)**

8) *интегрирования* - **(0.05 балла)**

9) Вызова многочлена как функции (вычисление значения в точке) - **(0.03 балла)**

**Комментарии:**

Для комплексных коэффициентов **(0.01 балла)** к каждому пункту.

Операции с числами также должны работать.

In [700]:
import numpy as np
import matplotlib.pyplot as plt

class Polynomial:
    
    def __init__(self, *coefficients):

        self.coefficients = list(coefficients)
        self.degree = len(self.coefficients) - 1
    
    # 1) сложения - (0.02 балла)
    def __add__(self, other):
        
        p1 = self.coefficients[::-1]
        p2 = other.coefficients[::-1]
        
        if len(p1) < len(p2):
            p1 += [0]*(len(p2) - len(p1))
        else:
            p2 += [0]*(len(p1) - len(p2))
        
        output = [sum(t) for t in zip(p1, p2)]
        
        return Polynomial(*output)
    
    # 2) вычитания - (0.02 балла)
    def __sub__(self, other):
        
        p1 = self.coefficients[::-1]
        p2 = other.coefficients[::-1]
        
        if len(p1) < len(p2):
            p1 += [0]*(len(p2) - len(p1))
        else:
            p2 += [0]*(len(p1) - len(p2))
        
        output = [t1 - t2 for t1, t2 in zip(p1, p2)]
        
        return Polynomial(*output)
       
    # 3) *умножения* - **(0.04 балла)**
    def __mul__(self, other):
        "Return self*val"
        
        p1 = self.coefficients
        p2 = other.coefficients
        output = [0]*(len(p1) + len(p2) - 1)
        
        for p1_deg, p1_coef in enumerate(p1):
            for p2_deg, p2_coef in enumerate(p2):
                output[p1_deg + p2_deg] += p1_coef*p2_coef
        return Polynomial(*output)

    # 3a) *быстрого умножения* (алгоритм Карацубы или быстрое преобразование Фурье) - **(+0.25 балла)**

    # 4) *деления* - **(0.05 балла)**

    # 5) *возведения в степень* - **(0.02 балла)** | *возведения в степень* через быстрое возведение в степень за log - **(0.04 балла)**
    def __pow__(self, power):
       
        if power == 0:
            return Polynomial(0)
        if power == 1:
            return self
    
        tmp = self
        for i in range(2, power + 1):
            tmp *= self

        return tmp
    
    # 6) представления многочлена в человеческом виде - (0.02 балла)
    def __repr__(self):
        
        output = ""
        degree = len(self.coefficients) - 1
        
        for i, coef in enumerate(self.coefficients):
            if self.degree - i > 1:
                x_degree = "x^" + str(self.degree - i)  
            elif self.degree - i == 1:
                x_degree = "x"
            else:
                x_degree = ""
                
            if abs(coef) == 1 and i != self.degree:
                str_coef = ""  
            else:
                str_coef = str(abs(coef))
            
            if coef < 0:
                output += " - " +  str_coef + x_degree
            elif coef > 0 and i == 0:
                output += str_coef + x_degree
            elif coef > 0 and i > 0:
                output += " + " +  str_coef + x_degree
            else:
                continue
            
        return output
    
    # 7) дифференцирования - (0.05 балла)
    def derivative(self):
        
        new_coefs = []
        degree = len(self.coefficients) - 1
        
        for i in range(len(self.coefficients)-1):
            
            new_coefs.append(self.coefficients[i] * degree)
            degree -= 1
            
        return Polynomial(*new_coefs)
    
    # 8) *интегрирования* - **(0.05 балла)**
    def integral(self, left, right, n=1000):
        h = (right - left)/n
        output = 0
        for i in range(n):
            output += self((left + h/2) + i*h)
            
        output *= h
        return output

    # 9) Вызова многочлена как функции (вычисление значения в точке) - (0.03 балла)
    def __call__(self, x):  
        
        output = 0
        for d, coef in enumerate(self.coefficients[::-1]):
            output += coef*x**d
        return output 

### Задача 3 [Аналог range] (0.05 балла)

**Условие:**

Реализуйте итератор с поведением, аналогичным range.

In [701]:
def my_range(*args):
    
    if len(args) == 0:
        start, end, step = 0, 0, 1
    elif len(args) == 1:
        start, end, step = 0, args[0], 1
    elif len(args) == 2:
        start, end, step = args[0], args[1], 1
    elif len(args) == 3:
        start, end, step = args[0], args[1], args[2]
    else:
        raise TypeError(f"range expected at most {3} arguments, got {len(args)}")
        
    i = start
    
    while i < end - 1:
        i += step
        yield i

### Задача 4 [Primary Key] (0.05 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте Primary Key - свойства первичного ключа из PostgreSQL.

### Задача 5 [PositiveSmallIntegerField] (0.03 балла)

**Условие:**

С помощью механизма дескрипторов реализуйте тип данных PositiveSmallIntegerField - поле, принимающее значения от 0 до 32767.

### Задача 6 [Timer] (0.02 балла)

**Условие:**

Реализовать контекстный менеджер, который выводит время, проведенное в нём.

In [377]:
from time import time

class Timer(object):
    def __enter__(self):
        self.start = time()
    def __exit__(self, type, value, traceback):
        self.end = time()
        print(f"Time: {self.end - self.start}")

with Timer():
    a = [i for i in range(100)]

Time: 9.059906005859375e-06
